In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/My Drive/BTP-SEM-8/VideoAnomalyDetection/preprocess")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys  
sys.path.append('../')

In [4]:
from c3d.c3d import *
import imageio
from io import BytesIO
from keras.utils import to_categorical,Sequence
import numpy as np
import os
import parameters as params
import pickle
from utils.array_util import *
import zipfile
import cv2

Using TensorFlow backend.


In [5]:
c3d = c3d_feature_extractor()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








In [0]:
def c3d_features_from_frames(frames):
    global c3d
    clips = sliding_window(frames, params.frame_count, params.frame_count)
    preprocessed_clips=[]
    for i, clip in enumerate(clips):
        clip = np.array(clip)
        if len(clip) < params.frame_count:   #for ignoring the last clip with less than 16frames
            continue

        clip = preprocess_input(clip)
        preprocessed_clips.append(clip)
    # print(np.array(preprocessed_clips).shape)

    return c3d.predict(np.array(preprocessed_clips))

In [0]:
def extract_preprocess_image(archive, filename):
    global c3d
    video = archive.read(filename)
    vid = imageio.get_reader(BytesIO(video), 'ffmpeg')
    frames=[]
    predictions = np.empty((1,4096))
    try:
        for num, image in enumerate(vid.iter_data()): 
            frames.append(image)
            if len(frames) > 6400: # 400*16
                preds = c3d_features_from_frames(frames)
                predictions = np.concatenate((predictions, preds), axis=0)
                frames = []
    except:
        pass
    
    if len(frames) >= 16:
        preds = c3d_features_from_frames(frames)
        predictions = np.concatenate((predictions, preds), axis=0)
    return predictions[1:]

    """

    # print("Shape of frames extracted", np.array(frames).shape)
    #Now shape of frames[] is [no of frames, 240, 320, 3]

    #Before doing c3d preprocessing, we need to split this into clips of 16 frames each
    clips = sliding_window(frames, params.frame_count, params.frame_count)
    print("Number of clips after sliding window", len(clips))
    # print("Shape of each clip after sliding window", np.array(clips[0]).shape)
    #shape of clips is (noOfFrames/16, 16, 240, 320, 3) 

    #Now we need to c3d preprocess each clip
    predictions = np.empty((1,4096))
    preprocessed_clips=[]
    for i, clip in enumerate(clips):
        clip = np.array(clip)
        if len(clip) < params.frame_count:   #for ignoring the last clip with less than 16frames
            continue

        clip = preprocess_input(clip)
        preprocessed_clips.append(clip)
        if len(preprocessed_clips) > 400:
            #print ("In processed clips:", i)
            preprocessed_clips = np.array(preprocessed_clips)
            pred = c3d.predict(preprocessed_clips)
            predictions = np.concatenate((predictions,pred),axis=0)
            preprocessed_clips = []
        # print("Pre-processed clip : ", i)
    if len(preprocessed_clips)!=0:
        pred = c3d.predict(np.array(preprocessed_clips))
        predictions = np.concatenate((predictions,pred),axis=0)
    # print("Shape after preprocessing", np.array(preprocessed_clips).shape)
    return predictions[1:]
    """

In [0]:
#For Image preloaded image features
class My_Custom_Generator_Features(Sequence) :
    def __init__(self,filenames, labels, batch_size, archive) :
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        self.archive = archive
    def __len__(self) :
        return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(np.int)
    
    def __getitem__(self, idx) :
        print("index:", idx)
        batch_x = self.filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        #batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
        videos_x = np.empty((1,16,112,112,3))
        for counter, video in enumerate(batch_x):
            print("counter:", counter)
            op = extract_preprocess_image(self.archive, video)
            print(op.shape)
            videos_x = np.concatenate((videos_x, op), axis=0)
    
        batch_y = np.zeros((len(videos_x)-1, 4096))
        return videos_x[1:], batch_y

In [0]:
def getFeatures(archive, fname, countVal = 0):
    c3d = c3d_feature_extractor()
    names = []
    fileNames=[]
    for name in archive.namelist():
        if name.endswith('.mp4'):
            fileNames.append(name)
            name = name.split('/')
            name = "/".join(name[1:])
            names.append(name)
    print("no of filenames: ", len(fileNames))
    predictions = []
    # if countVal != 0:
        # predictions = pickle.load( open( "../Dataset/" + fname + ".pkl", "rb" ) )
    for counter, name in enumerate(fileNames[countVal:], countVal):

        print("counter:",counter)
        op = extract_preprocess_image(archive, name)
        predictions.append(op)
        if counter % 50 == 0:
            pickle.dump( predictions, open( "../Dataset/" + fname + "-" + str(counter) + ".pkl", "wb" ))
            predictions = []
    pickle.dump( predictions, open( "../Dataset/" + fname + "-" + str(len(fileNames)-1)+ ".pkl", "wb" ))
    return predictions, names

In [0]:
#saving c3d features into .txt files
def convert_to_txt( rgb_features_matrix, filenames, path_to_save, fname):
    N = rgb_features_matrix.shape[0]
    print("No of videos: ", N)

    all_video_features=[]

    for i in range(N):
        one_video_features = rgb_features_matrix[i,:,:]
        one_video_features = one_video_features.flatten().astype(np.float32) #converting [32,4096] into [32*4096]
        # print("skajdkasj",one_video_features)
        # print("dtype:", one_video_features.dtype)

        if filenames !=None:

            filename = filenames[i].replace('/', '-')
            filename = filename.replace('.mp4', '.txt')
            

            new_filename = os.path.join(path_to_save, filename) 
        # np.savetxt(new_filename, one_video_features, newline=' ') #saving these into a text file
        all_video_features.append(one_video_features)

    pickle.dump(all_video_features, open( fname + ".pkl", "wb" ))
    return

In [0]:
#function to split the processed videos into training and testing based on the .txt file which contains the names of testing videos
def train_test_spilt_c3d(c3d_matrix, filename_matrix, test_split_file): #test_split_file is path of the file which contains names of testing videos
    N = len(c3d_matrix)
    print("Total number of elements given for split:", N)
    
    f = open(test_split_file, 'r')
    testing_file_names = f.read().splitlines()
    # testing_file_names = f.readlines()
    f.close()

    # testing_file_names = np.loadtxt(test_split_file) #loading these names into a numpy array
    print("No of testing elements found:", len(testing_file_names))
    print(testing_file_names)
    train_c3d_array=[]
    test_c3d_array=[]
    train_filenames=[]
    test_filenames=[]
    
    for i in range(N):
        if(filename_matrix[i] in testing_file_names):  #checking if the i'th filename exists in the np array containing test file names
            test_c3d_array.append(c3d_matrix[i,:,:])
            test_filenames.append(filename_matrix[i])
        else:
            train_c3d_array.append(c3d_matrix[i,:,:])
            train_filenames.append(filename_matrix[i])

    return train_c3d_array, test_c3d_array, train_filenames, test_filenames

In [0]:
def save_to_text_features(pred, names, fname_train, fname_test):
  features_bag=[]

  for pred in preds:
    bag = interpolate(pred, params.features_per_bag)
    features_bag.append(bag)

  features_bag = np.array(features_bag)
  print("Shape of features bag", features_bag.shape)

  test_split_file="../Dataset/UCF_Crimes-Train-Test-Split/Anomaly_Detection_splits/Anomaly_Test.txt"
  train_c3d_array, test_c3d_array, train_filenames, test_filenames = train_test_spilt_c3d(features_bag, names, test_split_file)

  train_c3d_array = np.array(train_c3d_array)
  test_c3d_array = np.array(test_c3d_array)
  train_filenames = np.array(train_filenames)
  test_filenames = np.array(test_filenames)
  print("train_c3d_array.shape", train_c3d_array.shape)
  print("test_c3d_array,shape", test_c3d_array.shape)
  print("train_filenames.shape", train_filenames.shape)
  print("test_filenames.shape", test_filenames.shape)

  path_for_training_c3d_features="../c3d_predictions/Anomaly/Train_features"
  fname = "../c3d_predictions/Anomaly/Train_features/Anomaly1" 
  convert_to_txt(train_c3d_array, train_filenames, path_for_training_c3d_features, fname_train)

  path_for_testing_c3d_features="../c3d_predictions/Anomaly/Test_features"
  fname = "../c3d_predictions/Anomaly/Test_features/Anomaly1" 
  convert_to_txt(test_c3d_array, test_filenames, path_for_testing_c3d_features, fname_test)

In [0]:
def new_save_to_text_features(pred, names, fname_train):
  features_bag=[]

  for pred in preds:
    bag = interpolate(pred, params.features_per_bag)
    features_bag.append(bag)

  features_bag = np.array(features_bag)
  print("Shape of features bag", features_bag.shape)

  path_for_training_c3d_features="../c3d_predictions/Anomaly/Train_features"
  # fname = "../c3d_predictions/Anomaly/Train_features/Anomaly1" 
  convert_to_txt(features_bag, names, path_for_training_c3d_features, fname_train)

  

In [0]:
def combine_features(path, fname):
    preds = np.empty((1,32*4096))
    for i in os.listdir(path):
        pred = pickle.load( open( os.path.join(path,i), "rb" ) )
        preds = np.concatenate((preds,np.array(pred)), axis = 0)
    
    preds = preds[1:]
    print(preds.shape)
    print(path + " " + fname)
    pickle.dump( preds, open( os.path.join(path,fname) + ".pkl", "wb" ))

In [0]:
archive = zipfile.ZipFile('../Dataset/AnomalyVideos/AnomalyVideos1.zip','r')
preds,names = getFeatures(archive, "AnomalyVideos1")
#shape of preds=[no of videos, no of clips, 4096]
#shape of names =[no of videos]
#Step1 : change shape of preds to [no of videos, no of clips, 4096] using interpolate
#Step 2: splitting into train test
#Step 3: save into txt

counter: 0


/usr/local/lib/python3.6/dist-packages/imageio/core/format.py:411: UserWarning: Could not read last frame of <file>.
  warn("Could not read last frame of %s." % uri)


counter: 1
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
counter: 20
counter: 21
counter: 22
counter: 23
counter: 24
counter: 25
counter: 26
counter: 27
counter: 28
counter: 29
counter: 30
counter: 31
counter: 32
counter: 33
counter: 34
counter: 35
counter: 36
counter: 37
counter: 38
counter: 39
counter: 40
counter: 41
counter: 42
counter: 43
counter: 44
counter: 45
counter: 46
counter: 47
counter: 48
counter: 49
counter: 50
counter: 51
counter: 52
counter: 53
counter: 54
counter: 55
counter: 56
counter: 57
counter: 58
counter: 59
counter: 60
counter: 61
counter: 62
counter: 63
counter: 64
counter: 65
counter: 66
counter: 67
counter: 68
counter: 69
counter: 70
counter: 71
counter: 72
counter: 73
counter: 74
counter: 75
counter: 76
counter: 77
counter: 78
counter: 79
counter: 80
counter: 81
counter: 82
counter: 83
counter: 84
c

In [0]:
preds = np.array(preds)
names= np.array(names)
print("Shape of preds:", np.array(preds).shape)
print("Shape of preds[0]:", preds[0].shape)
print("Shape of preds[11]:", preds[11].shape)

#shape of preds=[no of videos, no of clips, 4096]
print("Shape of names:", np.array(names).shape)
#shape of names =[no of videos]

Shape of preds: (200,)
Shape of preds[0]: (86, 4096)
Shape of preds[11]: (1779, 4096)
Shape of names: (200,)


In [0]:
#Step1 : change shape of preds to [no of videos, no of clips, 4096] using interpolate
features_bag=[]

for pred in preds:
  bag = interpolate(pred, params.features_per_bag)
  features_bag.append(bag)

features_bag = np.array(features_bag)
print("Shape of features bag", features_bag.shape)
#shape of features_bag =[no of videos, 32, 4096] 

feature_size 4096
interpolation_indicies [ 0.  3.  5.  8. 11. 13. 16. 19. 21. 24. 27. 29. 32. 35. 37. 40. 42. 45.
 48. 50. 53. 56. 58. 61. 64. 66. 69. 72. 74. 77. 80. 82. 85.]
feature_size 4096
interpolation_indicies [  0.   3.   7.  10.  14.  17.  21.  24.  28.  31.  35.  38.  42.  45.
  49.  52.  56.  59.  62.  66.  69.  73.  76.  80.  83.  87.  90.  94.
  97. 101. 104. 108. 111.]
feature_size 4096
interpolation_indicies [  0.  10.  20.  30.  40.  50.  60.  70.  80.  90. 100. 110. 120. 130.
 140. 150. 160. 169. 179. 189. 199. 209. 219. 229. 239. 249. 259. 269.
 279. 289. 299. 309. 319.]
feature_size 4096
interpolation_indicies [  0.   5.  11.  16.  21.  26.  32.  37.  42.  48.  53.  58.  63.  69.
  74.  79.  84.  90.  95. 100. 106. 111. 116. 121. 127. 132. 137. 143.
 148. 153. 158. 164. 169.]
feature_size 4096
interpolation_indicies [  0.   7.  13.  20.  26.  33.  40.  46.  53.  60.  66.  73.  80.  86.
  93.  99. 106. 113. 119. 126. 132. 139. 146. 152. 159. 166. 172. 179.
 186. 192. 

In [0]:
print(names[15])
print(features_bag[15].shape)
print(features_bag[15])

Abuse/Abuse026_x264.mp4
(32, 4096)
[[0.00165074 0.01031937 0.06375679 ... 0.         0.         0.        ]
 [0.         0.0251886  0.05927483 ... 0.         0.         0.        ]
 [0.         0.01690006 0.06056022 ... 0.         0.00077664 0.02312388]
 ...
 [0.         0.03766897 0.04558364 ... 0.         0.         0.04725234]
 [0.         0.00931773 0.07226696 ... 0.         0.         0.04326748]
 [0.         0.00422203 0.07346436 ... 0.         0.         0.02964902]]


In [0]:
#Step 2: splitting into train test
test_split_file="../Dataset/UCF_Crimes-Train-Test-Split/Anomaly_Detection_splits/Anomaly_Test.txt"
train_c3d_array, test_c3d_array, train_filenames, test_filenames = train_test_spilt_c3d(features_bag, names, test_split_file)

train_c3d_array = np.array(train_c3d_array)
test_c3d_array = np.array(test_c3d_array)
train_filenames = np.array(train_filenames)
test_filenames = np.array(test_filenames)
print("train_c3d_array.shape", train_c3d_array.shape)
print("test_c3d_array,shape", test_c3d_array.shape)
print("train_filenames.shape", train_filenames.shape)
print("test_filenames.shape", test_filenames.shape)

# print("train_c3d_array[0].shape", train_c3d_array[0].shape)
# print("test_c3d_array[0].shape", test_c3d_array[0].shape)
# print("train_filenames[0].shape", train_filenames[0].shape)
# print("test_filenames[0].shape", test_filenames[0].shape)

Total number of elements given for split: 200
No of testing elements found: 290
['Abuse/Abuse028_x264.mp4', 'Abuse/Abuse030_x264.mp4', 'Arrest/Arrest001_x264.mp4', 'Arrest/Arrest007_x264.mp4', 'Arrest/Arrest024_x264.mp4', 'Arrest/Arrest030_x264.mp4', 'Arrest/Arrest039_x264.mp4', 'Arson/Arson007_x264.mp4', 'Arson/Arson009_x264.mp4', 'Arson/Arson010_x264.mp4', 'Arson/Arson011_x264.mp4', 'Arson/Arson016_x264.mp4', 'Arson/Arson018_x264.mp4', 'Arson/Arson022_x264.mp4', 'Arson/Arson035_x264.mp4', 'Arson/Arson041_x264.mp4', 'Assault/Assault006_x264.mp4', 'Assault/Assault010_x264.mp4', 'Assault/Assault011_x264.mp4', 'Burglary/Burglary005_x264.mp4', 'Burglary/Burglary017_x264.mp4', 'Burglary/Burglary018_x264.mp4', 'Burglary/Burglary021_x264.mp4', 'Burglary/Burglary024_x264.mp4', 'Burglary/Burglary032_x264.mp4', 'Burglary/Burglary033_x264.mp4', 'Burglary/Burglary035_x264.mp4', 'Burglary/Burglary037_x264.mp4', 'Burglary/Burglary061_x264.mp4', 'Burglary/Burglary076_x264.mp4', 'Burglary/Burglary079

In [0]:
#Step 3: save into txt

path_for_training_c3d_features="../c3d_predictions/Anomaly/Train_features"
fname = "../c3d_predictions/Anomaly/Train_features/Anomaly1" 
convert_to_txt(train_c3d_array, train_filenames, path_for_training_c3d_features, fname)

path_for_testing_c3d_features="../c3d_predictions/Anomaly/Test_features"
fname = "../c3d_predictions/Anomaly/Test_features/Anomaly1" 
convert_to_txt(test_c3d_array, test_filenames, path_for_testing_c3d_features, fname)

No of videos:  181
No of videos:  19


In [0]:
archive = zipfile.ZipFile('../Dataset/AnomalyVideos/AnomalyVideos3.zip','r')
preds,names = getFeatures(archive, "AnomalyVideos3")
save_to_text_features(preds, names,"../c3d_predictions/Anomaly/Train_features/Anomaly3", "../c3d_predictions/Anomaly/Test_features/Anomaly3" )

no of filenames:  350
counter: 0
counter: 1
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12


/usr/local/lib/python3.6/dist-packages/imageio/core/format.py:411: UserWarning: Could not read last frame of <file>.
  warn("Could not read last frame of %s." % uri)


counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
counter: 20
counter: 21
counter: 22
counter: 23
counter: 24
counter: 25
counter: 26
counter: 27
counter: 28
counter: 29
counter: 30
counter: 31
counter: 32
counter: 33
counter: 34
counter: 35
counter: 36
counter: 37
counter: 38
counter: 39
counter: 40
counter: 41
counter: 42
counter: 43
counter: 44
counter: 45
counter: 46
counter: 47
counter: 48
counter: 49
counter: 50
counter: 51
counter: 52
counter: 53
counter: 54
counter: 55
counter: 56
counter: 57
counter: 58
counter: 59
counter: 60
counter: 61
counter: 62
counter: 63
counter: 64
counter: 65
counter: 66
counter: 67
counter: 68
counter: 69
counter: 70
counter: 71
counter: 72
counter: 73
counter: 74
counter: 75
counter: 76
counter: 77
counter: 78
counter: 79
counter: 80
counter: 81
counter: 82
counter: 83
counter: 84
counter: 85
counter: 86
counter: 87
counter: 88
counter: 89
counter: 90
counter: 91
counter: 92
counter: 93
counter: 94
counter: 95
coun

In [0]:
archive = zipfile.ZipFile('../Dataset/AnomalyVideos/AnomalyVideos4.zip','r')
preds,names = getFeatures(archive, "AnomalyVideos4")
save_to_text_features(preds, names,"../c3d_predictions/Anomaly/Train_features/Anomaly4", "../c3d_predictions/Anomaly/Test_features/Anomaly4" )

no of filenames:  200
counter: 0
counter: 1


/usr/local/lib/python3.6/dist-packages/imageio/core/format.py:411: UserWarning: Could not read last frame of <file>.
  warn("Could not read last frame of %s." % uri)


counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
counter: 20
counter: 21
counter: 22
counter: 23
counter: 24
counter: 25
counter: 26
counter: 27
counter: 28
counter: 29
counter: 30
counter: 31
counter: 32
counter: 33
counter: 34
counter: 35
counter: 36
counter: 37
counter: 38
counter: 39
counter: 40
counter: 41
counter: 42
counter: 43
counter: 44
counter: 45
counter: 46
counter: 47
counter: 48
counter: 49
counter: 50
counter: 51
counter: 52
counter: 53
counter: 54
counter: 55
counter: 56
counter: 57
counter: 58
counter: 59
counter: 60
counter: 61
counter: 62
counter: 63
counter: 64
counter: 65
counter: 66
counter: 67
counter: 68
counter: 69
counter: 70
counter: 71
counter: 72
counter: 73
counter: 74
counter: 75
counter: 76
counter: 77
counter: 78
counter: 79
counter: 80
counter: 81
counter: 82
counter: 83
counter: 84
counter: 85


In [0]:
archive = zipfile.ZipFile('../Dataset/AnomalyVideos/Testing_Normal_Videos.zip','r')
preds,names = getFeatures(archive, "Testing_Normal_Videos")
save_to_text_features(preds, names,"../c3d_predictions/Normal/Train_features/Testing_Normal_Videos", "../c3d_predictions/Normal/Test_features/Testing_Normal_Videos" )

no of filenames:  150
counter: 0
counter: 1


/usr/local/lib/python3.6/dist-packages/imageio/core/format.py:411: UserWarning: Could not read last frame of <file>.
  warn("Could not read last frame of %s." % uri)


counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
counter: 20
counter: 21
counter: 22
counter: 23
counter: 24
counter: 25
counter: 26
counter: 27
counter: 28
counter: 29
counter: 30
counter: 31
counter: 32
counter: 33
counter: 34
counter: 35
counter: 36
counter: 37
counter: 38
counter: 39
counter: 40
counter: 41
counter: 42
counter: 43
counter: 44
counter: 45
counter: 46
counter: 47
counter: 48
counter: 49
counter: 50
counter: 51
counter: 52
counter: 53
counter: 54
counter: 55
counter: 56
counter: 57
counter: 58
counter: 59
counter: 60
counter: 61
counter: 62
counter: 63
counter: 64
counter: 65
counter: 66
counter: 67
counter: 68
counter: 69
counter: 70
counter: 71
counter: 72
counter: 73
counter: 74
counter: 75
counter: 76
counter: 77
counter: 78
counter: 79
counter: 80
counter: 81
counter: 82
counter: 83
counter: 84
counter: 85


In [0]:
archive = zipfile.ZipFile('../Dataset/AnomalyVideos/Training-Normal-Videos-Part-1.zip','r')
preds,names = getFeatures(archive, "Training-Normal-Videos-Part-1", 401)
# save_to_text_features(preds, names,"../c3d_predictions/Normal/Train_features/Training-Normal-Videos-Part-1", "../c3d_predictions/Normal/Test_features/Training-Normal-Videos-Part-1" )

no of filenames:  430
counter: 401
counter: 402
counter: 403
counter: 404
counter: 405
counter: 406


/usr/local/lib/python3.6/dist-packages/imageio/core/format.py:411: UserWarning: Could not read last frame of <file>.
  warn("Could not read last frame of %s." % uri)


counter: 407
counter: 408
counter: 409
counter: 410
counter: 411
counter: 412
counter: 413
counter: 414
counter: 415
counter: 416
counter: 417
counter: 418
counter: 419
counter: 420
counter: 421
counter: 422
counter: 423
counter: 424
counter: 425
counter: 426
counter: 427
counter: 428
counter: 429


In [0]:
archive = zipfile.ZipFile('../Dataset/AnomalyVideos/Training-Normal-Videos-Part-2.zip','r')
preds,names = getFeatures(archive, "Training-Normal-Videos-Part-2",351)
# save_to_text_features(preds, names,"../c3d_predictions/Normal/Train_features/Training-Normal-Videos-Part-2", "../c3d_predictions/Normal/Test_features/Training-Normal-Videos-Part-2" )

no of filenames:  370
counter: 351
counter: 352
counter: 353


/usr/local/lib/python3.6/dist-packages/imageio/core/format.py:411: UserWarning: Could not read last frame of <file>.
  warn("Could not read last frame of %s." % uri)


counter: 354
counter: 355
counter: 356
counter: 357
counter: 358
counter: 359
counter: 360
counter: 361
counter: 362
counter: 363
counter: 364
counter: 365
counter: 366
counter: 367
counter: 368
counter: 369


In [0]:
path = '../Dataset'
counter = 0
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and 'Copy of Training-Normal-Videos-Part-2' in i:
        #preds = pickle.load( open( os.path.join(path,i), "rb" ) )
        #new_save_to_text_features(preds, None, "../c3d_predictions/Normal/Train_features/Training-Normal-Videos-Part-2" +"-"+ str(counter))
        #counter+=1
        print(i)
        

Copy of Training-Normal-Videos-Part-2-200.pkl
Copy of Training-Normal-Videos-Part-2-150.pkl
Copy of Training-Normal-Videos-Part-2-50.pkl
Copy of Training-Normal-Videos-Part-2-250.pkl
Copy of Training-Normal-Videos-Part-2-100.pkl
Copy of Training-Normal-Videos-Part-2-300.pkl
Copy of Training-Normal-Videos-Part-2-369.pkl
Copy of Training-Normal-Videos-Part-2-350.pkl
Copy of Training-Normal-Videos-Part-2-0.pkl


In [23]:
path = '../Dataset'
counter = 0
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and 'Training-Normal-Videos-Part-1' in i:
        if i == "Copy of Training-Normal-Videos-Part-1.pkl":
          continue
        preds = pickle.load( open( os.path.join(path,i), "rb" ) )
        new_save_to_text_features(preds, None, "../c3d_predictions/Normal/Train_features/Training-Normal-Videos-Part-1" +"-"+ str(counter))
        counter+=1
        # print(i)

feature_size 4096
interpolation_indicies [ 0.  1.  2.  3.  4.  6.  7.  8.  9. 10. 11. 12. 14. 15. 16. 17. 18. 19.
 20. 21. 22. 24. 25. 26. 27. 28. 29. 30. 32. 33. 34. 35. 36.]
feature_size 4096
interpolation_indicies [  0.   3.   7.  10.  14.  17.  20.  24.  27.  30.  34.  37.  40.  44.
  47.  51.  54.  57.  61.  64.  68.  71.  74.  78.  81.  84.  88.  91.
  94.  98. 101. 105. 108.]
feature_size 4096
interpolation_indicies [  0.   6.  12.  19.  25.  31.  37.  44.  50.  56.  62.  68.  75.  81.
  87.  93. 100. 106. 112. 118. 124. 131. 137. 143. 149. 155. 162. 168.
 174. 180. 187. 193. 199.]
feature_size 4096
interpolation_indicies [  0.   4.   8.  12.  16.  20.  24.  28.  32.  35.  39.  43.  47.  51.
  55.  59.  63.  67.  71.  75.  79.  83.  87.  91.  94.  98. 102. 106.
 110. 114. 118. 122. 126.]
feature_size 4096
interpolation_indicies [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 22. 23. 24. 25. 26. 27. 28. 29. 30.]
feature_size 4096


In [0]:
combine_features("../c3d_predictions/Anomaly/Train_features", "train_features")

(810, 131072)
../c3d_predictions/Anomaly/Train_features train_features


In [0]:
combine_features("../c3d_predictions/Anomaly/Test_features", "test_features")

(140, 131072)
../c3d_predictions/Anomaly/Test_features test_features


In [24]:
combine_features("../c3d_predictions/Normal/Train_features", "train_features")

(800, 131072)
../c3d_predictions/Normal/Train_features train_features


In [0]:
!ls ../c3d_predictions/Anomaly/Train_features

Anomaly1.pkl  Anomaly2.pkl  Anomaly3.pkl  Anomaly4.pkl


In [0]:
preds = pickle.load( open( os.path.join("../c3d_predictions/Anomaly/Train_features","Anomaly1.pkl"), "rb" ) )

In [0]:
preds = pickle.load( open( os.path.join("../Dataset/Training-Normal-Videos-Part-1-100.pkl"), "rb" ) )

In [20]:
print(np.array(preds).shape)

(101,)


In [21]:
preds[14].shape

(54, 4096)